In [1]:
# Spam filter in python
import pandas as pd
import numpy as np
import matplotlib as mpt
import matplotlib.pyplot as plt
import seaborn as sns 
import statsmodels.api as sm
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler  
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from numpy import array
from sklearn.model_selection import train_test_split

In [2]:
# For reference: this is taken from https://github.com/obieda01/Hands-on-machine-learning-scikit-learn-tensorflow-Book/blob/master/03_classification.ipynb
import os
import tarfile
from six.moves import urllib

DOWNLOAD_ROOT = "http://spamassassin.apache.org/old/publiccorpus/"
HAM_URL = DOWNLOAD_ROOT + "20030228_easy_ham.tar.bz2"
SPAM_URL = DOWNLOAD_ROOT + "20030228_spam.tar.bz2"
SPAM_PATH = os.path.join("datasets", "spam")

def fetch_spam_data(spam_url=SPAM_URL, spam_path=SPAM_PATH):
    if not os.path.isdir(spam_path):
        os.makedirs(spam_path)
    for filename, url in (("ham.tar.bz2", HAM_URL), ("spam.tar.bz2", SPAM_URL)):
        path = os.path.join(spam_path, filename)
        if not os.path.isfile(path):
            urllib.request.urlretrieve(url, path)
        tar_bz2_file = tarfile.open(path)
        tar_bz2_file.extractall(path=SPAM_PATH)
        tar_bz2_file.close()


HAM_DIR = os.path.join(SPAM_PATH, "easy_ham")
SPAM_DIR = os.path.join(SPAM_PATH, "spam")
ham_filenames = [name for name in sorted(os.listdir(HAM_DIR)) if len(name) > 20]
spam_filenames = [name for name in sorted(os.listdir(SPAM_DIR)) if len(name) > 20]

import email
import email.policy

def load_email(is_spam, filename, spam_path=SPAM_PATH):
    directory = "spam" if is_spam else "easy_ham"
    with open(os.path.join(spam_path, directory, filename), "rb") as f:
        return email.parser.BytesParser(policy=email.policy.default).parse(f)
    
    
ham_emails = [load_email(is_spam=False, filename=name) for name in ham_filenames]
spam_emails = [load_email(is_spam=True, filename=name) for name in spam_filenames]

In [3]:
import email
from bs4 import BeautifulSoup
q= str(ham_emails[13].get_payload())
#body = BeautifulSoup(ham_emails[13].get_content()).get_text()
q=BeautifulSoup(q).get_text()
if(q == '[, ]'):
    q="multipart"



In [4]:
from sklearn.feature_extraction.text import CountVectorizer
import re
text = re.sub(r'^https?:\/\/.*[\r\n]*', 'URL', ham_emails[0].get_content(), flags=re.MULTILINE)

length = len(ham_emails)
i=1
while i < length:
    newTxt=str(ham_emails[i].get_payload())
    newTxt=BeautifulSoup(newTxt).get_text()
    if(newTxt== '[, ]'):
        newTxt="multipart"
        
    newTxt = re.sub(r'^https?:\/\/.*[\r\n]*', 'URL', newTxt,flags=re.MULTILINE)
    newTxt = re.sub(r"http\S+", "URL", newTxt)
    newTxt = re.sub(r'^ftp?:\/\/.*[\r\n]*', 'FTP', newTxt,flags=re.MULTILINE)
    text=text+newTxt
    text=text.strip()
    i+=1
    




/Users/fadelalshammasi/opt/anaconda3/lib/python3.7/site-packages/bs4/__init__.py:357: UserWarning: "http://www.post-gazette.com/columnists/20020905brian5.asp



" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup


In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer 

textLst= text.split("\n")
tfidf_vect=TfidfVectorizer(stop_words='english')
X=tfidf_vect.fit_transform(textLst)
X.shape

(78256, 27065)

In [6]:
vocab=tfidf_vect.get_feature_names()
dfHam=pd.DataFrame(X.toarray(), columns=vocab)
dfHam['isSpam']=0
dfHam

,00,000,0000,000000,000000000,0008,00085k,000th,001,002,...,zzzzteana,³bigfoot,³care,écrit,écrite,éliminé,élite,était,être,isSpam
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78251,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
78252,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
78253,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
78254,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [7]:
text = re.sub(r'^https?:\/\/.*[\r\n]*', 'URL', spam_emails[0].get_content(), flags=re.MULTILINE)

length = len(spam_emails)
i=1
while i < length:
    newTxt=str(spam_emails[i].get_payload())
    newTxt=BeautifulSoup(newTxt).get_text()
    if(newTxt== '[, ]'):
        newTxt="multipart"
        
    newTxt = re.sub(r'^https?:\/\/.*[\r\n]*', 'URL', newTxt,flags=re.MULTILINE)
    newTxt = re.sub(r"http\S+", "URL", newTxt)
    newTxt = re.sub(r'^ftp?:\/\/.*[\r\n]*', 'URL', newTxt,flags=re.MULTILINE)
    text=text+newTxt
    text=text.strip()
    i+=1
    





In [8]:
textLst= text.split("\n")
tfidf_vect=TfidfVectorizer(stop_words='english')
X=tfidf_vect.fit_transform(textLst)
X.shape

vocab=tfidf_vect.get_feature_names()
dfSpam=pd.DataFrame(X.toarray(), columns=vocab)
dfSpam['isSpam']=1
dfSpam

,00,000,000000,000033,000080,0000a0,0000ff,0001pt,0003,000e,...,현재,호황을,홈쇼핑의,확실한,활황을,훨씬,힘입어,０福壕w路推,ｉt的技,isSpam
0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,1
1,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,1
2,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,1
3,0.236701,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,1
4,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27502,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,1
27503,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,1
27504,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.510741,0.0,1
27505,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,1


In [9]:
dfHam

,00,000,0000,000000,000000000,0008,00085k,000th,001,002,...,zzzzteana,³bigfoot,³care,écrit,écrite,éliminé,élite,était,être,isSpam
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78251,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
78252,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
78253,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
78254,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [10]:
dfAll=pd.concat([dfHam, dfSpam], join="inner")

In [11]:
df1 = dfAll.drop(['isSpam'], axis=1)
X=df1
y=dfAll["isSpam"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1996)

In [12]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn import svm

model=LogisticRegression() # tried out other algorthims but Logistic Regression was the best
model.fit(X_train,y_train)


/Users/fadelalshammasi/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [13]:
accuracy=model.score(X_test,y_test)

In [14]:
accuracy

0.7953481775634662

In [15]:
#labels = model.predict(test.data)
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import confusion_matrix, classification_report
y_pred = model.predict(X_test)
my_conf = confusion_matrix(y_test,y_pred)
my_conf

array([[15335,   317],
       [ 4012,  1489]])

In [16]:
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.79      0.98      0.88     15652
           1       0.82      0.27      0.41      5501

    accuracy                           0.80     21153
   macro avg       0.81      0.63      0.64     21153
weighted avg       0.80      0.80      0.75     21153

